In [1]:
import pandas as pd

df = pd.read_parquet('1_21_25_2ndUpdatedSchoolNames.parquet')

In [2]:
df.shape

(358104, 20)

In [3]:
df.loc[df['Name'] == 'Filip Duric', ['ID', 'Power_Tot']] = [839394, 23.63]
df = df[~((df['Name'] == 'Rachel Flaherty') & (df['School'] == 'Sacred Heart University'))]
df = df[~((df['Name'] == 'Jason Allen') & (df['School'] == 'Missouri State University'))]
df = df[~((df['Name'] == 'Andrew Johnson') & (df['School'] == 'Missouri State University'))]

In [4]:

df2 = pd.read_csv('1_21_25_correcteddups.csv')

# Step 2: For each row in df2, check for matches in df
for index2, row2 in df2.iterrows():
    # Step 3: For each row in df, match the 'Name' and 'School' columns
    mask = (df['Name'] == row2['Name']) & (df['School'] == row2['School'])
    
    # Step 4: Update the 'ID' in df if a match is found
    df.loc[mask, 'ID'] = row2['ID']

In [5]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from IPython.display import clear_output
def getPowerIndex(swimmer_ID):
	swimmer_url = 'https://swimcloud.com/swimmer/' + str(swimmer_ID)

	url = requests.get(swimmer_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
	url.encoding = 'utf-8'
	
	if url.status_code == 404:
		raise ValueError(f"The swimmer {swimmer_ID} was not found")

	soup = bs(url.text, 'html.parser')
	

	data_array = soup.find_all('li', {'class' : 'c-list-bar__item'}) # returns the container items containing headers for recruiting stats
	for d in data_array:
		if d.find(class_='c-list-bar__subheader').get('title') == 'Power index':
				return float(d.find(class_='c-list-bar__description').text.strip())
	if soup.find('h1', {'class' : 'c-toolbar__title'}).text.strip() == None:
		print("ERROR PROCESSING")
		time.sleep(30)
		return -1.0

	swimmer_name = soup.find('h1', {'class' : 'c-toolbar__title'}).text.strip()

	swimmer_name_url = 'https://swimcloud.com/recruiting/rankings/?name=' + swimmer_name.replace(' ', '+')

	name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})

	name_url.encoding = 'utf-8'

	name_soup = bs(name_url.text, 'html.parser')
	if name_soup.find('tbody').find_all('tr') == None:
		time.sleep(30)
		return -1.0

	swimmer_list = name_soup.find('tbody').find_all('tr')
	i=0
	
	for swimmer in swimmer_list:
		#check if this is the correct swimmer by looking to see if the link matches their id
		name = swimmer.find(class_='u-text-semi')
		swimmer_link = name['href'] if name else ''
		if(swimmer_link == "/swimmer/" + str(swimmer_ID)):
			#a power index was found for the specified swimmer_ID!
			idx = float(name_soup.find_all('td', {'class' : 'u-text-end'})[i].text.strip())
			print( idx )
			return idx
		i = i + 1

	return -1 #if no swimmer is found with the correct swimmer ID #, return -1

In [6]:
arr1 = df2.ID.values

In [7]:
df = df[~df['ID'].isin(df2['ID'])]

In [8]:
df.shape

(354960, 20)

In [9]:
getPowerIndex(1826414)

-1

In [10]:
import numpy as np
index = np.where(arr1 == 1826414)[0]
index
arr2[101]

NameError: name 'arr2' is not defined

In [ ]:
df[df.Name=='Nicholas Edwards']

In [11]:
df.to_parquet('CLEANED_SWIM_DATA.parquet')